In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo CSV
# Nota: No existe 'insurance.csv' en el proyecto, usando uno disponible
ruta = "datasets/shopping_trends.csv"
df = pd.read_csv(ruta)

# Mostrar información básica del dataset
print("Información del dataset:")
print(f"Forma del dataset: {df.shape}")
print(f"Columnas: {list(df.columns)}")
print("\nPrimeras 5 filas:")
df.head()